In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Activation Functions

In [ ]:
class ActivationFunction:
    def __init__(self):
        pass
    
    def forward(self, x):
        pass
    
    def backward(self, x):
        pass

class Sigmoid(ActivationFunction):
    def forward(self, x):
        return 1 / (1 + np.exp(-x))
    
    def backward(self, x):
        return x * (1 - x)

class Tanh(ActivationFunction):
    def forward(self, x):
        return np.tanh(x)
    
    def backward(self, x):
        return 1 - x**2

class ReLU(ActivationFunction):
    def forward(self, x):
        return np.maximum(0, x)
    
    def backward(self, x):
        return (x > 0).astype(float)


# Optimizers

In [ ]:
class Optimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
    
    def update(self, layer):
        pass
    
class BatchGradientDescent(Optimizer):
    def update(self, layer, gradient_weights, gradient_biases):
        layer.weights -= self.learning_rate * gradient_weights
        layer.biases -= self.learning_rate * gradient_biases


class StochasticGradientDescent(Optimizer):
    def update(self, layer, gradient_weights, gradient_biases):
        layer.weights -= self.learning_rate * gradient_weights
        layer.biases -= self.learning_rate * gradient_biases

class MiniBatchGradientDescent(Optimizer):
    def __init__(self, learning_rate, batch_size):
        super().__init__(learning_rate)
        self.batch_size = batch_size
    
    def update(self, layer, gradient_weights, gradient_biases):
        layer.weights -= self.learning_rate * gradient_weights / self.batch_size
        layer.biases -= self.learning_rate * gradient_biases / self.batch_size


class Adam(Optimizer):
    def __init__(self, learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8):
        super().__init__(learning_rate)
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = 0
        self.v = 0
        self.t = 0

    def update(self, layer):
        self.t += 1
        layer.gradient_weights = layer.gradient_weights + self.epsilon
        layer.gradient_biases = layer.gradient_biases + self.epsilon
        self.m = self.beta1 * self.m + (1 - self.beta1) * layer.gradient_weights
        self.v = self.beta2 * self.v + (1 - self.beta2) * (layer.gradient_weights ** 2)
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        layer.weights -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
        layer.biases -= self.learning_rate * layer.gradient_biases / (np.sqrt(v_hat) + self.epsilon)


# Perceptron

In [ ]:
class Perceptron:
    def __init__(self, input_size):
        self.weights = np.random.rand(input_size)
        self.biases = np.random.rand(1)
        self.activation = None
        self.gradient_weights = None
        self.gradient_biases = None
    
    def set_activation(self, activation):
        self.activation = activation
    
    def forward(self, x):
        z = np.dot(x, self.weights) + self.biases
        return self.activation.forward(z)
    
    def backward(self, x, delta):
        self.gradient_weights = np.dot(x.T, delta)
        self.gradient_biases = np.sum(delta, axis=0)
        return np.dot(delta, self.weights.T)


# Perceptron Layer

In [ ]:
class PerceptronLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.perceptrons = [Perceptron(input_size) for _ in range(output_size)]
    
    def set_activation(self, activation):
        for perceptron in self.perceptrons:
            perceptron.set_activation(activation)
    
    def forward(self, x):
        return np.array([perceptron.forward(x) for perceptron in self.perceptrons])
    
    def backward(self, x, delta):
        return np.array([perceptron.backward(x, delta) for perceptron in self.perceptrons])


# Multi Layer Perceptron

In [ ]:
class MLPClassifier:
    def __init__(self, input_size, hidden_layer_sizes, output_size, learning_rate, activation, optimizer):
        self.layers = []
        self.hidden_layer_sizes = hidden_layer_sizes
        self.learning_rate = learning_rate
        self.activation = activation
        self.optimizer = optimizer
        
        # Create input layer
        input_layer = PerceptronLayer(input_size, hidden_layer_sizes[0])
        input_layer.set_activation(activation)
        self.layers.append(input_layer)
        
        # Create hidden layers
        for i in range(1, len(hidden_layer_sizes)):
            hidden_layer = PerceptronLayer(hidden_layer_sizes[i - 1], hidden_layer_sizes[i])
            hidden_layer.set_activation(activation)
            self.layers.append(hidden_layer)
        
        # Create output layer
        output_layer = PerceptronLayer(hidden_layer_sizes[-1], output_size)
        output_layer.set_activation(activation)
        self.layers.append(output_layer)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, x, y):
        # Backpropagation
        error = y - x
        for i in range(len(self.layers) - 1, -1, -1):
            error = self.layers[i].backward(x, error)
    
    def train(self, X, y, epochs):
        for epoch in range(epochs):
            for i in range(len(X)):
                x = X[i]
                target = y[i]
                prediction = self.forward(x)
                self.backward(prediction, target)
                for layer in self.layers:
                    self.optimizer.update(layer)


# Given By ChatGPT

In [ ]:
import numpy as np

# Activation Functions
class ActivationFunction:
    def __init__(self):
        pass
    
    def forward(self, x):
        pass
    
    def backward(self, x):
        pass

class Sigmoid(ActivationFunction):
    def forward(self, x):
        return 1 / (1 + np.exp(-x))
    
    def backward(self, x):
        return x * (1 - x)

class Tanh(ActivationFunction):
    def forward(self, x):
        return np.tanh(x)
    
    def backward(self, x):
        return 1 - x**2

class ReLU(ActivationFunction):
    def forward(self, x):
        return np.maximum(0, x)
    
    def backward(self, x):
        return (x > 0).astype(float)

# Optimizers
class Optimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
    
    def update(self, layer):
        pass

class BatchGradientDescent(Optimizer):
    def update(self, layer):
        layer.weights -= self.learning_rate * layer.gradient_weights
        layer.biases -= self.learning_rate * layer.gradient_biases

class StochasticGradientDescent(Optimizer):
    def update(self, layer):
        layer.weights -= self.learning_rate * layer.gradient_weights
        layer.biases -= self.learning_rate * layer.gradient_biases

class MiniBatchGradientDescent(Optimizer):
    def __init__(self, learning_rate, batch_size):
        super().__init__(learning_rate)
        self.batch_size = batch_size
    
    def update(self, layer):
        layer.weights -= self.learning_rate * layer.gradient_weights / self.batch_size
        layer.biases -= self.learning_rate * layer.gradient_biases / self.batch_size

class Adam(Optimizer):
    def __init__(self, learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8):
        super().__init__(learning_rate)
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = 0
        self.v = 0
        self.t = 0

    def update(self, layer):
        self.t += 1
        self.m = self.beta1 * self.m + (1 - self.beta1) * layer.gradient_weights
        self.v = self.beta2 * self.v + (1 - self.beta2) * (layer.gradient_weights ** 2)
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        layer.weights -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
        layer.biases -= self.learning_rate * layer.gradient_biases / (np.sqrt(v_hat) + self.epsilon)

# Perceptron
class Perceptron:
    def __init__(self, input_size):
        self.weights = np.random.rand(input_size)
        self.biases = np.random.rand(1)
        self.activation = None
        self.gradient_weights = None
        self.gradient_biases = None
    
    def set_activation(self, activation):
        self.activation = activation
    
    def forward(self, x):
        z = np.dot(x, self.weights) + self.biases
        return self.activation.forward(z)
    
    def backward(self, x, delta):
        self.gradient_weights = np.dot(x.T, delta)
        self.gradient_biases = np.sum(delta, axis=0)
        return np.dot(delta, self.weights.T)

# Perceptron Layer
class PerceptronLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.perceptrons = [Perceptron(input_size) for _ in range(output_size)]
    
    def set_activation(self, activation):
        for perceptron in self.perceptrons:
            perceptron.set_activation(activation)
    
    def forward(self, x):
        return np.array([perceptron.forward(x) for perceptron in self.perceptrons])
    
    def backward(self, x, delta):
        return np.array([perceptron.backward(x, delta) for perceptron in self.perceptrons])

# Multi Layer Perceptron
class MLPClassifier:
    def __init__(self, input_size, hidden_layer_sizes, output_size, learning_rate, activation, optimizer):
        self.layers = []
        self.hidden_layer_sizes = hidden_layer_sizes
        self.learning_rate = learning_rate
        self.activation = activation
        self.optimizer = optimizer
        
        # Create input layer
        input_layer = PerceptronLayer(input_size, hidden_layer_sizes[0])
        input_layer.set_activation(activation)
        self.layers.append(input_layer)
        
        # Create hidden layers
        for i in range(1, len(hidden_layer_sizes)):
            hidden_layer = PerceptronLayer(hidden_layer_sizes[i - 1], hidden_layer_sizes[i])
            hidden_layer.set_activation(activation)
            self.layers.append(hidden_layer)
        
        # Create output layer
        output_layer = PerceptronLayer(hidden_layer_sizes[-1], output_size)
        output_layer.set_activation(activation)
        self.layers.append(output_layer)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, x, y):
        # Backpropagation
        error = y - x
        for i in range(len(self.layers) - 1, -1, -1):
            error = self.layers[i].backward(x, error)
    
    def train(self, X, y, epochs):
        for epoch in range(epochs):
            for i in range(len(X)):
                x = X[i]
                target = y[i]
                prediction = self.forward(x)
                self.backward(prediction, target)
                for layer in self.layers:
                    self.optimizer.update(layer)
